# Tesla Stock SQL Analysis
## Applied Data Science Capstone Project

**Objetivo**: Realizar análisis avanzado de datos de Tesla usando SQL

**Áreas de análisis SQL**:
- Consultas de rendimiento temporal
- Análisis de patrones de trading
- Indicadores técnicos con SQL
- Análisis de volatilidad
- Consultas de correlación y tendencias

In [22]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Librerías importadas exitosamente")
print("📊 Listo para análisis SQL de datos de Tesla")

✅ Librerías importadas exitosamente
📊 Listo para análisis SQL de datos de Tesla


## 1. Configurar Base de Datos SQLite

In [19]:
# Configurar conexión a base de datos SQLite
def setup_database():
    """
    Configurar base de datos SQLite y cargar datos de Tesla
    """
    tesla_df = None
    conn = None
    
    try:
        # Intentar cargar datos desde archivo limpio
        tesla_df = pd.read_csv('data/clean/tesla_final_dataset.csv', index_col=0, parse_dates=True)
        print(f"✅ Datos cargados desde archivo: {tesla_df.shape}")
    
    except FileNotFoundError:
        print("⚠️ Archivo de datos no encontrado. Creando datos de ejemplo...")
        try:
            import yfinance as yf
            tesla_df = yf.download('TSLA', start='2020-01-01', end='2024-01-01', progress=False)
            
            # Normalizar columnas si es necesario
            if isinstance(tesla_df.columns, pd.MultiIndex):
                tesla_df.columns = tesla_df.columns.get_level_values(0)
            
            tesla_df.columns = [str(col).title() for col in tesla_df.columns]
            tesla_df['Daily_Return'] = tesla_df['Close'].pct_change()
            tesla_df = tesla_df.dropna()
            
            print(f"✅ Datos descargados con yfinance: {tesla_df.shape}")
            
        except Exception as e:
            print(f"❌ Error con yfinance: {e}")
            print("🔧 Creando datos sintéticos...")
            
            # Crear datos sintéticos
            dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
            np.random.seed(42)
            n_days = len(dates)
            
            price_changes = np.random.normal(0.001, 0.03, n_days)
            prices = 100 * np.exp(np.cumsum(price_changes))
            
            tesla_df = pd.DataFrame({
                'Open': prices * np.random.uniform(0.98, 1.02, n_days),
                'High': prices * np.random.uniform(1.00, 1.05, n_days),
                'Low': prices * np.random.uniform(0.95, 1.00, n_days),
                'Close': prices,
                'Volume': np.random.randint(10000000, 100000000, n_days),
                'Daily_Return': np.random.normal(0.001, 0.03, n_days)
            }, index=dates)
            
            tesla_df = tesla_df.dropna()
            print(f"✅ Datos sintéticos creados: {tesla_df.shape}")
    
    if tesla_df is not None:
        try:
            # Crear conexión SQLite en memoria
            conn = sqlite3.connect(':memory:')
            
            # Preparar DataFrame para SQL
            tesla_sql = tesla_df.reset_index()
            tesla_sql.columns = [col.replace(' ', '_') for col in tesla_sql.columns]
            
            # Asegurar que tenemos columnas básicas
            required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
            for col in required_columns:
                if col not in tesla_sql.columns and col.lower() in [c.lower() for c in tesla_sql.columns]:
                    # Encontrar la columna con nombre similar
                    actual_col = [c for c in tesla_sql.columns if c.lower() == col.lower()][0]
                    tesla_sql = tesla_sql.rename(columns={actual_col: col})
            
            # Agregar columnas calculadas si no existen
            if 'Daily_Return' not in tesla_sql.columns:
                tesla_sql['Daily_Return'] = tesla_sql['Close'].pct_change()
            
            # Agregar más características para análisis SQL
            tesla_sql['Price_Range'] = tesla_sql['High'] - tesla_sql['Low']
            tesla_sql['Price_Change'] = tesla_sql['Close'] - tesla_sql['Open']
            tesla_sql['Year'] = pd.to_datetime(tesla_sql['Date']).dt.year
            tesla_sql['Month'] = pd.to_datetime(tesla_sql['Date']).dt.month
            tesla_sql['DayOfWeek'] = pd.to_datetime(tesla_sql['Date']).dt.dayofweek
            tesla_sql['Quarter'] = pd.to_datetime(tesla_sql['Date']).dt.quarter
            
            # Cargar datos a SQLite
            tesla_sql.to_sql('tesla_stock', conn, index=False, if_exists='replace')
            
            print(f"✅ Base de datos SQLite configurada exitosamente")
            print(f"📊 Tabla 'tesla_stock' creada con {len(tesla_sql)} filas")
            print(f"📅 Período: {tesla_sql['Date'].min()} a {tesla_sql['Date'].max()}")
            
            # Mostrar esquema de la tabla
            schema_query = "PRAGMA table_info(tesla_stock)"
            schema = pd.read_sql_query(schema_query, conn)
            print(f"\n📋 ESQUEMA DE LA TABLA:")
            for _, row in schema.iterrows():
                print(f"  • {row['name']}: {row['type']}")
            
            return conn, tesla_sql
            
        except Exception as e:
            print(f"❌ Error configurando base de datos: {e}")
            if conn:
                conn.close()
            return None, None
    
    return None, None

# Ejecutar configuración
conn, tesla_data = setup_database()

if conn:
    print("\n🎉 ¡Base de datos lista para análisis SQL!")
else:
    print("\n❌ No se pudo configurar la base de datos.")

⚠️ Archivo de datos no encontrado. Creando datos de ejemplo...
✅ Datos descargados con yfinance: (1005, 6)
✅ Base de datos SQLite configurada exitosamente
📊 Tabla 'tesla_stock' creada con 1005 filas
📅 Período: 2020-01-03 00:00:00 a 2023-12-29 00:00:00

📋 ESQUEMA DE LA TABLA:
  • Date: TIMESTAMP
  • Close: REAL
  • High: REAL
  • Low: REAL
  • Open: REAL
  • Volume: INTEGER
  • Daily_Return: REAL
  • Price_Range: REAL
  • Price_Change: REAL
  • Year: INTEGER
  • Month: INTEGER
  • DayOfWeek: INTEGER
  • Quarter: INTEGER

🎉 ¡Base de datos lista para análisis SQL!


## 2. Consultas Básicas de Exploración

In [20]:
# Consultas básicas de exploración
def basic_sql_exploration(conn):
    """
    Realizar consultas básicas para explorar los datos
    """
    if conn is None:
        print("❌ No hay conexión a la base de datos")
        return
    
    print("📊 CONSULTAS BÁSICAS DE EXPLORACIÓN")
    print("=" * 40)
    
    # 1. Estadísticas básicas
    basic_stats = """
    SELECT 
        COUNT(*) as total_records,
        MIN(Date) as start_date,
        MAX(Date) as end_date,
        ROUND(MIN(Close), 2) as min_price,
        ROUND(MAX(Close), 2) as max_price,
        ROUND(AVG(Close), 2) as avg_price,
        ROUND(AVG(Volume), 0) as avg_volume
    FROM tesla_stock
    """
    
    stats_result = pd.read_sql_query(basic_stats, conn)
    print("\n📈 ESTADÍSTICAS GENERALES:")
    for col in stats_result.columns:
        value = stats_result[col].iloc[0]
        print(f"  • {col.replace('_', ' ').title()}: {value}")
    
    # 2. Top 10 días con mayor volumen
    high_volume = """
    SELECT 
        Date,
        ROUND(Close, 2) as Close_Price,
        Volume,
        ROUND(Daily_Return * 100, 2) as Daily_Return_Pct
    FROM tesla_stock
    ORDER BY Volume DESC
    LIMIT 10
    """
    
    volume_result = pd.read_sql_query(high_volume, conn)
    print("\n📊 TOP 10 DÍAS CON MAYOR VOLUMEN:")
    print(volume_result.to_string(index=False))
    
    # 3. Mayores ganancias diarias
    biggest_gains = """
    SELECT 
        Date,
        ROUND(Close, 2) as Close_Price,
        ROUND(Daily_Return * 100, 2) as Daily_Return_Pct,
        'Ganancia' as Move_Type
    FROM tesla_stock
    WHERE Daily_Return IS NOT NULL
    ORDER BY Daily_Return DESC
    LIMIT 5
    """
    
    gains_result = pd.read_sql_query(biggest_gains, conn)
    print("\n🎯 TOP 5 MAYORES GANANCIAS DIARIAS:")
    print(gains_result.to_string(index=False))
    
    # 4. Mayores pérdidas diarias  
    biggest_losses = """
    SELECT 
        Date,
        ROUND(Close, 2) as Close_Price,
        ROUND(Daily_Return * 100, 2) as Daily_Return_Pct,
        'Pérdida' as Move_Type
    FROM tesla_stock
    WHERE Daily_Return IS NOT NULL
    ORDER BY Daily_Return ASC
    LIMIT 5
    """
    
    losses_result = pd.read_sql_query(biggest_losses, conn)
    print("\n🎯 TOP 5 MAYORES PÉRDIDAS DIARIAS:")
    print(losses_result.to_string(index=False))
    
    return stats_result, volume_result, gains_result, losses_result

# Ejecutar consultas básicas
if conn:
    basic_results = basic_sql_exploration(conn)
else:
    print("⚠️ No se pueden ejecutar consultas sin conexión a la base de datos")

📊 CONSULTAS BÁSICAS DE EXPLORACIÓN

📈 ESTADÍSTICAS GENERALES:
  • Total Records: 1005
  • Start Date: 2020-01-03 00:00:00
  • End Date: 2023-12-29 00:00:00
  • Min Price: 24.08
  • Max Price: 409.97
  • Avg Price: 209.31
  • Avg Volume: 133216673.0

📊 TOP 10 DÍAS CON MAYOR VOLUMEN:
               Date  Close_Price    Volume  Daily_Return_Pct
2020-02-04 00:00:00        59.14 914082000             13.73
2020-02-05 00:00:00        48.98 726357000            -17.18
2020-02-03 00:00:00        52.00 705975000             19.89
2020-12-18 00:00:00       231.67 666378600              5.96
2020-02-06 00:00:00        49.93 598212000              1.94
2020-07-13 00:00:00        99.80 584781000             -3.08
2020-05-01 00:00:00        46.75 487977000            -10.30
2020-01-22 00:00:00        37.97 470535000              4.09
2020-01-08 00:00:00        32.81 467164500              4.92
2020-04-14 00:00:00        47.33 458647500              9.05

🎯 TOP 5 MAYORES GANANCIAS DIARIAS:
          

## 3. Análisis Temporal con SQL

In [23]:
# Análisis temporal usando SQL
def temporal_analysis_sql(conn):
    """
    Análisis temporal usando consultas SQL avanzadas
    """
    if conn is None:
        print("❌ No hay conexión a la base de datos")
        return
    
    print("📅 ANÁLISIS TEMPORAL CON SQL")
    print("=" * 35)
    
    # 1. Rendimiento por año - versión simplificada compatible con SQLite
    yearly_performance = """
    WITH yearly_data AS (
        SELECT 
            Year,
            Date,
            Close,
            Daily_Return,
            ROW_NUMBER() OVER (PARTITION BY Year ORDER BY Date) as first_day,
            ROW_NUMBER() OVER (PARTITION BY Year ORDER BY Date DESC) as last_day
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    ),
    yearly_summary AS (
        SELECT 
            Year,
            MIN(CASE WHEN first_day = 1 THEN Close END) as start_price,
            MIN(CASE WHEN last_day = 1 THEN Close END) as end_price,
            AVG(Daily_Return) as avg_daily_return,
            COUNT(*) as trading_days
        FROM yearly_data
        GROUP BY Year
    )
    SELECT 
        Year,
        ROUND(start_price, 2) as Start_Price,
        ROUND(end_price, 2) as End_Price,
        ROUND((end_price - start_price) / start_price * 100, 2) as Annual_Return_Pct,
        ROUND(avg_daily_return * 100, 3) as Avg_Daily_Return_Pct,
        trading_days as Trading_Days
    FROM yearly_summary
    ORDER BY Year
    """
    
    yearly_result = pd.read_sql_query(yearly_performance, conn)
    print("\n📊 RENDIMIENTO ANUAL:")
    print(yearly_result.to_string(index=False))
    
    # 2. Análisis por mes - simplificado sin volatilidad
    monthly_analysis = """
    SELECT 
        Month,
        CASE Month
            WHEN 1 THEN 'Enero' WHEN 2 THEN 'Febrero' WHEN 3 THEN 'Marzo'
            WHEN 4 THEN 'Abril' WHEN 5 THEN 'Mayo' WHEN 6 THEN 'Junio'
            WHEN 7 THEN 'Julio' WHEN 8 THEN 'Agosto' WHEN 9 THEN 'Septiembre'
            WHEN 10 THEN 'Octubre' WHEN 11 THEN 'Noviembre' WHEN 12 THEN 'Diciembre'
        END as Month_Name,
        COUNT(*) as Trading_Days,
        ROUND(AVG(Daily_Return) * 100, 3) as Avg_Daily_Return_Pct,
        ROUND(AVG(ABS(Daily_Return)) * 100, 3) as Avg_Abs_Return_Pct,
        SUM(CASE WHEN Daily_Return > 0 THEN 1 ELSE 0 END) as Positive_Days,
        ROUND(AVG(Volume), 0) as Avg_Volume
    FROM tesla_stock
    WHERE Daily_Return IS NOT NULL
    GROUP BY Month
    ORDER BY Month
    """
    
    monthly_result = pd.read_sql_query(monthly_analysis, conn)
    print("\n📅 ANÁLISIS MENSUAL:")
    print(monthly_result.to_string(index=False))
    
    # 3. Análisis por día de la semana - simplificado
    dow_analysis = """
    SELECT 
        DayOfWeek,
        CASE DayOfWeek
            WHEN 0 THEN 'Lunes' WHEN 1 THEN 'Martes' WHEN 2 THEN 'Miércoles'
            WHEN 3 THEN 'Jueves' WHEN 4 THEN 'Viernes'
        END as Day_Name,
        COUNT(*) as Trading_Days,
        ROUND(AVG(Daily_Return) * 100, 3) as Avg_Daily_Return_Pct,
        ROUND(AVG(ABS(Daily_Return)) * 100, 3) as Avg_Abs_Return_Pct,
        SUM(CASE WHEN Daily_Return > 0 THEN 1 ELSE 0 END) as Positive_Days,
        ROUND(AVG(Volume), 0) as Avg_Volume
    FROM tesla_stock
    WHERE Daily_Return IS NOT NULL AND DayOfWeek < 5  -- Solo días laborales
    GROUP BY DayOfWeek
    ORDER BY DayOfWeek
    """
    
    dow_result = pd.read_sql_query(dow_analysis, conn)
    print("\n📊 ANÁLISIS POR DÍA DE LA SEMANA:")
    print(dow_result.to_string(index=False))
    
    return yearly_result, monthly_result, dow_result

# Ejecutar análisis temporal
if conn:
    temporal_results = temporal_analysis_sql(conn)
else:
    print("⚠️ No se puede ejecutar análisis temporal sin conexión a la base de datos")

📅 ANÁLISIS TEMPORAL CON SQL

📊 RENDIMIENTO ANUAL:
 Year  Start_Price  End_Price  Annual_Return_Pct  Avg_Daily_Return_Pct  Trading_Days
 2020        29.53     235.22             696.45                 0.997           252
 2021       243.26     352.26              44.81                 0.219           252
 2022       399.93     123.18             -69.20                -0.329           251
 2023       108.10     248.48             129.86                 0.339           250

📅 ANÁLISIS MENSUAL:
 Month Month_Name  Trading_Days  Avg_Daily_Return_Pct  Avg_Abs_Return_Pct  Positive_Days  Avg_Volume
     1      Enero            79                 1.073               3.756             47 175897294.0
     2    Febrero            76                 0.089               3.686             39 179912467.0
     3      Marzo            91                 0.136               4.107             44 155849176.0
     4      Abril            81                 0.114               3.367             41 145480465.0

## 4. Análisis de Patrones de Trading

In [24]:
# Análisis de patrones de trading con SQL
def trading_patterns_analysis(conn):
    """
    Analizar patrones de trading usando SQL
    """
    if conn is None:
        print("❌ No hay conexión a la base de datos")
        return
    
    print("🔍 ANÁLISIS DE PATRONES DE TRADING")
    print("=" * 40)
    
    # 1. Análisis de rachas ganadoras y perdedoras
    streak_analysis = """
    WITH daily_direction AS (
        SELECT 
            Date,
            Close,
            Daily_Return,
            CASE 
                WHEN Daily_Return > 0 THEN 1
                WHEN Daily_Return < 0 THEN -1
                ELSE 0
            END as direction,
            ROW_NUMBER() OVER (ORDER BY Date) as rn
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    ),
    streak_groups AS (
        SELECT *,
            rn - ROW_NUMBER() OVER (PARTITION BY direction ORDER BY Date) as streak_group
        FROM daily_direction
        WHERE direction != 0
    ),
    streaks AS (
        SELECT 
            direction,
            streak_group,
            COUNT(*) as streak_length,
            MIN(Date) as streak_start,
            MAX(Date) as streak_end
        FROM streak_groups
        GROUP BY direction, streak_group
    )
    SELECT 
        CASE 
            WHEN direction = 1 THEN 'Racha Ganadora'
            ELSE 'Racha Perdedora'
        END as Streak_Type,
        COUNT(*) as Total_Streaks,
        ROUND(AVG(streak_length), 1) as Avg_Length,
        MAX(streak_length) as Max_Length,
        MIN(streak_length) as Min_Length
    FROM streaks
    GROUP BY direction
    ORDER BY direction DESC
    """
    
    streak_result = pd.read_sql_query(streak_analysis, conn)
    print("\n🎯 ANÁLISIS DE RACHAS:")
    print(streak_result.to_string(index=False))
    
    # 2. Análisis de gaps (brechas de precios)
    gap_analysis = """
    WITH price_gaps AS (
        SELECT 
            Date,
            Open,
            Close,
            LAG(Close) OVER (ORDER BY Date) as prev_close,
            Open - LAG(Close) OVER (ORDER BY Date) as gap_amount,
            (Open - LAG(Close) OVER (ORDER BY Date)) / LAG(Close) OVER (ORDER BY Date) * 100 as gap_pct
        FROM tesla_stock
    )
    SELECT 
        CASE 
            WHEN ABS(gap_pct) >= 5 THEN 'Gap Grande (≥5%)'
            WHEN ABS(gap_pct) >= 2 THEN 'Gap Medio (2-5%)'
            WHEN ABS(gap_pct) >= 1 THEN 'Gap Pequeño (1-2%)'
            ELSE 'Sin Gap (<1%)'
        END as Gap_Category,
        COUNT(*) as Frequency,
        ROUND(AVG(gap_pct), 2) as Avg_Gap_Pct,
        ROUND(MIN(gap_pct), 2) as Min_Gap_Pct,
        ROUND(MAX(gap_pct), 2) as Max_Gap_Pct
    FROM price_gaps
    WHERE gap_pct IS NOT NULL
    GROUP BY Gap_Category
    ORDER BY Frequency DESC
    """
    
    gap_result = pd.read_sql_query(gap_analysis, conn)
    print("\n📊 ANÁLISIS DE GAPS (BRECHAS):")
    print(gap_result.to_string(index=False))
    
    # 3. Análisis de volumen vs movimiento de precio
    volume_price_analysis = """
    WITH volume_categories AS (
        SELECT 
            Date,
            Volume,
            Daily_Return,
            ABS(Daily_Return) as abs_return,
            AVG(Volume) OVER () as avg_volume,
            CASE 
                WHEN Volume >= AVG(Volume) OVER () * 2 THEN 'Volumen Muy Alto (>2x)'
                WHEN Volume >= AVG(Volume) OVER () * 1.5 THEN 'Volumen Alto (1.5-2x)'
                WHEN Volume >= AVG(Volume) OVER () THEN 'Volumen Medio (1-1.5x)'
                ELSE 'Volumen Bajo (<1x)'
            END as volume_category
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    )
    SELECT 
        volume_category as Volume_Category,
        COUNT(*) as Trading_Days,
        ROUND(AVG(Daily_Return) * 100, 3) as Avg_Daily_Return_Pct,
        ROUND(AVG(abs_return) * 100, 3) as Avg_Abs_Return_Pct,
        ROUND(AVG(Volume), 0) as Avg_Volume
    FROM volume_categories
    GROUP BY volume_category
    ORDER BY Avg_Volume DESC
    """
    
    volume_result = pd.read_sql_query(volume_price_analysis, conn)
    print("\n📈 VOLUMEN VS MOVIMIENTO DE PRECIO:")
    print(volume_result.to_string(index=False))
    
    return streak_result, gap_result, volume_result

# Ejecutar análisis de patrones
if conn:
    pattern_results = trading_patterns_analysis(conn)
else:
    print("⚠️ No se puede ejecutar análisis de patrones sin conexión a la base de datos")

🔍 ANÁLISIS DE PATRONES DE TRADING

🎯 ANÁLISIS DE RACHAS:
    Streak_Type  Total_Streaks  Avg_Length  Max_Length  Min_Length
 Racha Ganadora            249         2.2          13           1
Racha Perdedora            248         1.9           7           1

📊 ANÁLISIS DE GAPS (BRECHAS):
      Gap_Category  Frequency  Avg_Gap_Pct  Min_Gap_Pct  Max_Gap_Pct
     Sin Gap (<1%)        439         0.08        -0.99         0.99
Gap Pequeño (1-2%)        278         0.17        -1.98         1.99
  Gap Medio (2-5%)        222         0.22        -4.98         4.90
  Gap Grande (≥5%)         65         1.74       -14.90        13.20

📈 VOLUMEN VS MOVIMIENTO DE PRECIO:
       Volume_Category  Trading_Days  Avg_Daily_Return_Pct  Avg_Abs_Return_Pct  Avg_Volume
Volumen Muy Alto (>2x)            74                 2.107               6.928 366956091.0
 Volumen Alto (1.5-2x)            87                 0.815               5.001 227947664.0
Volumen Medio (1-1.5x)           188                 0.37

## 5. Análisis de Volatilidad con SQL

In [25]:
# Análisis de volatilidad usando SQL
def volatility_analysis_sql(conn):
    """
    Análisis de volatilidad usando consultas SQL avanzadas
    """
    if conn is None:
        print("❌ No hay conexión a la base de datos")
        return
    
    print("🌊 ANÁLISIS DE VOLATILIDAD CON SQL")
    print("=" * 40)
    
    # 1. Volatilidad por períodos - simplificado sin SQRT
    volatility_periods = """
    WITH volatility_data AS (
        SELECT 
            Date,
            Daily_Return,
            ABS(Daily_Return) as abs_return,
            Daily_Return * Daily_Return as squared_return,
            -- Promedio de retornos absolutos móvil de 20 días (proxy de volatilidad)
            AVG(ABS(Daily_Return)) OVER (
                ORDER BY Date 
                ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
            ) as rolling_abs_return_20d
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    )
    SELECT 
        CASE 
            WHEN rolling_abs_return_20d >= 0.04 THEN 'Muy Alta (≥4%)'
            WHEN rolling_abs_return_20d >= 0.025 THEN 'Alta (2.5-4%)'
            WHEN rolling_abs_return_20d >= 0.015 THEN 'Media (1.5-2.5%)'
            ELSE 'Baja (<1.5%)'
        END as Volatility_Category,
        COUNT(*) as Trading_Days,
        ROUND(AVG(rolling_abs_return_20d) * 100, 2) as Avg_Abs_Return_20d_Pct,
        ROUND(AVG(abs_return) * 100, 2) as Avg_Daily_Abs_Return_Pct,
        ROUND(MAX(abs_return) * 100, 2) as Max_Daily_Abs_Return_Pct
    FROM volatility_data
    WHERE rolling_abs_return_20d IS NOT NULL
    GROUP BY Volatility_Category
    ORDER BY Avg_Abs_Return_20d_Pct DESC
    """
    
    volatility_result = pd.read_sql_query(volatility_periods, conn)
    print("\n📊 CATEGORÍAS DE VOLATILIDAD:")
    print(volatility_result.to_string(index=False))
    
    # 2. Análisis de días de alta volatilidad - simplificado
    high_volatility_days = """
    WITH daily_volatility AS (
        SELECT 
            Date,
            Close,
            Daily_Return,
            ABS(Daily_Return) as abs_return,
            Volume,
            AVG(ABS(Daily_Return)) OVER () as overall_avg_abs_return
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    )
    SELECT 
        Date,
        ROUND(Close, 2) as Close_Price,
        ROUND(Daily_Return * 100, 2) as Daily_Return_Pct,
        Volume,
        ROUND(abs_return / overall_avg_abs_return, 1) as Volatility_Multiple
    FROM daily_volatility
    WHERE abs_return > overall_avg_abs_return * 2.5  -- Más de 2.5x el promedio
    ORDER BY abs_return DESC
    LIMIT 15
    """
    
    high_vol_result = pd.read_sql_query(high_volatility_days, conn)
    print("\n🔥 TOP 15 DÍAS DE MAYOR VOLATILIDAD:")
    print(high_vol_result.to_string(index=False))
    
    # 3. Volatilidad por trimestre y año - usando retornos absolutos
    quarterly_volatility = """
    SELECT 
        Year,
        Quarter,
        COUNT(*) as Trading_Days,
        ROUND(AVG(ABS(Daily_Return)) * 100, 2) as Avg_Abs_Return_Pct,
        ROUND(MAX(ABS(Daily_Return)) * 100, 2) as Max_Abs_Return_Pct,
        SUM(CASE WHEN ABS(Daily_Return) > 0.03 THEN 1 ELSE 0 END) as High_Volatility_Days
    FROM tesla_stock
    WHERE Daily_Return IS NOT NULL
    GROUP BY Year, Quarter
    ORDER BY Year, Quarter
    """
    
    quarterly_vol_result = pd.read_sql_query(quarterly_volatility, conn)
    print("\n📅 VOLATILIDAD POR TRIMESTRE:")
    print(quarterly_vol_result.to_string(index=False))
    
    return volatility_result, high_vol_result, quarterly_vol_result

# Ejecutar análisis de volatilidad
if conn:
    volatility_results = volatility_analysis_sql(conn)
else:
    print("⚠️ No se puede ejecutar análisis de volatilidad sin conexión a la base de datos")

🌊 ANÁLISIS DE VOLATILIDAD CON SQL

📊 CATEGORÍAS DE VOLATILIDAD:
Volatility_Category  Trading_Days  Avg_Abs_Return_20d_Pct  Avg_Daily_Abs_Return_Pct  Max_Daily_Abs_Return_Pct
     Muy Alta (≥4%)           176                    5.00                      5.00                     21.06
      Alta (2.5-4%)           505                    3.14                      3.11                     13.53
   Media (1.5-2.5%)           282                    2.07                      2.09                     12.66
       Baja (<1.5%)            42                    1.25                      1.23                      3.86

🔥 TOP 15 DÍAS DE MAYOR VOLATILIDAD:
               Date  Close_Price  Daily_Return_Pct    Volume  Volatility_Multiple
2020-09-08 00:00:00       110.07            -21.06 346397100                  6.8
2020-02-03 00:00:00        52.00             19.89 705975000                  6.5
2021-03-09 00:00:00       224.53             19.64 202569900                  6.4
2020-03-16 00:00:00  

## 6. Consultas de Correlación y Tendencias

In [26]:
# Análisis de correlaciones y tendencias con SQL
def correlation_trends_analysis(conn):
    """
    Análisis de correlaciones y tendencias usando SQL
    """
    if conn is None:
        print("❌ No hay conexión a la base de datos")
        return
    
    print("🔗 ANÁLISIS DE CORRELACIONES Y TENDENCIAS")
    print("=" * 45)
    
    # 1. Análisis de tendencias usando promedios móviles
    trend_analysis = """
    WITH moving_averages AS (
        SELECT 
            Date,
            Close,
            AVG(Close) OVER (
                ORDER BY Date 
                ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
            ) as ma_20,
            AVG(Close) OVER (
                ORDER BY Date 
                ROWS BETWEEN 49 PRECEDING AND CURRENT ROW
            ) as ma_50,
            Daily_Return
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    ),
    trend_signals AS (
        SELECT *,
            CASE 
                WHEN Close > ma_20 AND ma_20 > ma_50 THEN 'Tendencia Alcista Fuerte'
                WHEN Close > ma_20 AND ma_20 <= ma_50 THEN 'Tendencia Alcista Débil'
                WHEN Close <= ma_20 AND ma_20 > ma_50 THEN 'Tendencia Bajista Débil'
                WHEN Close <= ma_20 AND ma_20 <= ma_50 THEN 'Tendencia Bajista Fuerte'
                ELSE 'Lateral'
            END as trend_signal
        FROM moving_averages
        WHERE ma_50 IS NOT NULL
    )
    SELECT 
        trend_signal as Trend_Signal,
        COUNT(*) as Trading_Days,
        ROUND(AVG(Daily_Return) * 100, 3) as Avg_Daily_Return_Pct,
        ROUND(AVG(ABS(Daily_Return)) * 100, 3) as Avg_Abs_Return_Pct,
        SUM(CASE WHEN Daily_Return > 0 THEN 1 ELSE 0 END) as Positive_Days
    FROM trend_signals
    GROUP BY trend_signal
    ORDER BY Avg_Daily_Return_Pct DESC
    """
    
    trend_result = pd.read_sql_query(trend_analysis, conn)
    print("\n📈 ANÁLISIS DE TENDENCIAS:")
    print(trend_result.to_string(index=False))
    
    # 2. Correlación entre volumen y retornos
    volume_return_correlation = """
    WITH volume_buckets AS (
        SELECT 
            Date,
            Volume,
            Daily_Return,
            ABS(Daily_Return) as abs_return,
            NTILE(5) OVER (ORDER BY Volume) as volume_quintile
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL AND Volume IS NOT NULL
    )
    SELECT 
        volume_quintile as Volume_Quintile,
        COUNT(*) as Trading_Days,
        ROUND(AVG(Volume), 0) as Avg_Volume,
        ROUND(AVG(Daily_Return) * 100, 3) as Avg_Daily_Return_Pct,
        ROUND(AVG(abs_return) * 100, 3) as Avg_Abs_Return_Pct,
        ROUND(MIN(Volume), 0) as Min_Volume,
        ROUND(MAX(Volume), 0) as Max_Volume
    FROM volume_buckets
    GROUP BY volume_quintile
    ORDER BY volume_quintile
    """
    
    correlation_result = pd.read_sql_query(volume_return_correlation, conn)
    print("\n📊 CORRELACIÓN VOLUMEN-RETORNO (POR QUINTILES):")
    print(correlation_result.to_string(index=False))
    
    # 3. Análisis de momentum (retornos consecutivos)
    momentum_analysis = """
    WITH momentum_data AS (
        SELECT 
            Date,
            Daily_Return,
            LAG(Daily_Return, 1) OVER (ORDER BY Date) as prev_return_1,
            LAG(Daily_Return, 2) OVER (ORDER BY Date) as prev_return_2,
            LAG(Daily_Return, 3) OVER (ORDER BY Date) as prev_return_3,
            LEAD(Daily_Return, 1) OVER (ORDER BY Date) as next_return_1
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    )
    SELECT 
        'Retorno después de 3 días positivos' as Scenario,
        COUNT(*) as Occurrences,
        ROUND(AVG(next_return_1) * 100, 3) as Avg_Next_Return_Pct,
        SUM(CASE WHEN next_return_1 > 0 THEN 1 ELSE 0 END) as Positive_Next_Days
    FROM momentum_data
    WHERE prev_return_1 > 0 AND prev_return_2 > 0 AND prev_return_3 > 0
      AND next_return_1 IS NOT NULL
    
    UNION ALL
    
    SELECT 
        'Retorno después de 3 días negativos' as Scenario,
        COUNT(*) as Occurrences,
        ROUND(AVG(next_return_1) * 100, 3) as Avg_Next_Return_Pct,
        SUM(CASE WHEN next_return_1 > 0 THEN 1 ELSE 0 END) as Positive_Next_Days
    FROM momentum_data
    WHERE prev_return_1 < 0 AND prev_return_2 < 0 AND prev_return_3 < 0
      AND next_return_1 IS NOT NULL
    """
    
    momentum_result = pd.read_sql_query(momentum_analysis, conn)
    print("\n🎯 ANÁLISIS DE MOMENTUM:")
    print(momentum_result.to_string(index=False))
    
    return trend_result, correlation_result, momentum_result

# Ejecutar análisis de correlaciones
if conn:
    correlation_results = correlation_trends_analysis(conn)
else:
    print("⚠️ No se puede ejecutar análisis de correlaciones sin conexión a la base de datos")

🔗 ANÁLISIS DE CORRELACIONES Y TENDENCIAS

📈 ANÁLISIS DE TENDENCIAS:
            Trend_Signal  Trading_Days  Avg_Daily_Return_Pct  Avg_Abs_Return_Pct  Positive_Days
 Tendencia Alcista Débil           195                 1.723               2.961            128
Tendencia Alcista Fuerte           381                 1.027               2.868            244
Tendencia Bajista Fuerte           228                -0.581               3.534             96
 Tendencia Bajista Débil           201                -1.424               3.067             72

📊 CORRELACIÓN VOLUMEN-RETORNO (POR QUINTILES):
 Volume_Quintile  Trading_Days  Avg_Volume  Avg_Daily_Return_Pct  Avg_Abs_Return_Pct  Min_Volume  Max_Volume
               1           201  57191795.0                -0.132               1.567  29401800.0  70488600.0
               2           201  83697644.0                 0.025               2.696  70545400.0  95672100.0
               3           201 108198500.0                -0.032             

## 7. Resumen y Insights Clave del Análisis SQL

In [27]:
# Generar resumen de análisis SQL
def generate_sql_summary(conn):
    """
    Generar resumen comprehensivo del análisis SQL
    """
    if conn is None:
        print("❌ No hay conexión a la base de datos")
        return
    
    print("📊 RESUMEN DE ANÁLISIS SQL - TESLA STOCK")
    print("=" * 50)
    
    # Consulta resumen general - simplificada sin SQRT
    summary_query = """
    WITH overall_stats AS (
        SELECT 
            COUNT(*) as total_records,
            MIN(Date) as start_date,
            MAX(Date) as end_date,
            ROUND((MAX(Close) - MIN(Close)) / MIN(Close) * 100, 1) as total_price_change_pct,
            ROUND(AVG(Daily_Return) * 100, 3) as avg_daily_return_pct,
            ROUND(AVG(ABS(Daily_Return)) * 100, 3) as avg_abs_return_pct,
            SUM(CASE WHEN Daily_Return > 0 THEN 1 ELSE 0 END) as positive_days,
            SUM(CASE WHEN Daily_Return < 0 THEN 1 ELSE 0 END) as negative_days,
            ROUND(AVG(Volume), 0) as avg_volume,
            ROUND(MAX(Daily_Return) * 100, 2) as max_daily_gain_pct,
            ROUND(MIN(Daily_Return) * 100, 2) as max_daily_loss_pct
        FROM tesla_stock
        WHERE Daily_Return IS NOT NULL
    )
    SELECT * FROM overall_stats
    """
    
    summary_result = pd.read_sql_query(summary_query, conn)
    
    print("\n🔍 ESTADÍSTICAS GENERALES:")
    stats = summary_result.iloc[0]
    print(f"  • Período analizado: {stats['start_date']} a {stats['end_date']}")
    print(f"  • Total de registros: {stats['total_records']:,} días")
    print(f"  • Cambio total de precio: {stats['total_price_change_pct']}%")
    print(f"  • Retorno diario promedio: {stats['avg_daily_return_pct']}%")
    print(f"  • Retorno absoluto promedio: {stats['avg_abs_return_pct']}%")
    print(f"  • Días positivos: {stats['positive_days']} ({stats['positive_days']/stats['total_records']*100:.1f}%)")
    print(f"  • Días negativos: {stats['negative_days']} ({stats['negative_days']/stats['total_records']*100:.1f}%)")
    print(f"  • Volumen promedio: {stats['avg_volume']:,.0f} acciones")
    print(f"  • Máxima ganancia diaria: {stats['max_daily_gain_pct']}%")
    print(f"  • Máxima pérdida diaria: {stats['max_daily_loss_pct']}%")
    
    # Insights clave usando SQL
    insights_query = """
    WITH insights AS (
        SELECT 
            -- Mejor y peor mes
            (SELECT Month || ' (' || ROUND(AVG(Daily_Return) * 100, 2) || '%)' 
             FROM tesla_stock WHERE Daily_Return IS NOT NULL 
             GROUP BY Month ORDER BY AVG(Daily_Return) DESC LIMIT 1) as best_month,
            
            (SELECT Month || ' (' || ROUND(AVG(Daily_Return) * 100, 2) || '%)' 
             FROM tesla_stock WHERE Daily_Return IS NOT NULL 
             GROUP BY Month ORDER BY AVG(Daily_Return) ASC LIMIT 1) as worst_month,
            
            -- Mejor día de la semana
            (SELECT DayOfWeek || ' (' || ROUND(AVG(Daily_Return) * 100, 2) || '%)' 
             FROM tesla_stock WHERE Daily_Return IS NOT NULL AND DayOfWeek < 5
             GROUP BY DayOfWeek ORDER BY AVG(Daily_Return) DESC LIMIT 1) as best_day,
            
            -- Correlación volumen-volatilidad (aproximación)
            (SELECT COUNT(*) FROM tesla_stock 
             WHERE Volume > (SELECT AVG(Volume) FROM tesla_stock) 
               AND ABS(Daily_Return) > (SELECT AVG(ABS(Daily_Return)) FROM tesla_stock WHERE Daily_Return IS NOT NULL)) as high_vol_high_volatility,
            
            (SELECT COUNT(*) FROM tesla_stock 
             WHERE Volume > (SELECT AVG(Volume) FROM tesla_stock)) as high_volume_days
    )
    SELECT * FROM insights
    """
    
    insights_result = pd.read_sql_query(insights_query, conn)
    insights = insights_result.iloc[0]
    
    print("\n🎯 INSIGHTS CLAVE DEL ANÁLISIS SQL:")
    print(f"  • Mejor mes para invertir: {insights['best_month']}")
    print(f"  • Peor mes históricamente: {insights['worst_month']}")
    print(f"  • Mejor día de la semana: {insights['best_day']}")
    
    vol_correlation = insights['high_vol_high_volatility'] / insights['high_volume_days'] * 100
    print(f"  • Días de alto volumen con alta volatilidad: {vol_correlation:.1f}%")
    
    # Recomendaciones basadas en SQL
    print("\n💡 RECOMENDACIONES BASADAS EN ANÁLISIS SQL:")
    print("  • Los análisis de rachas muestran patrones de momentum")
    print("  • La volatilidad se concentra en períodos específicos")
    print("  • Existe correlación entre volumen alto y movimientos significativos")
    print("  • Los patrones estacionales son detectables con SQL")
    print("  • Las consultas complejas revelan insights ocultos en los datos")
    
    print("\n✅ ANÁLISIS SQL COMPLETADO")
    print("🚀 ¡Listo para modelado predictivo!")
    
    return summary_result, insights_result

# Ejecutar resumen final
if conn:
    final_summary = generate_sql_summary(conn)
else:
    print("⚠️ No se puede generar resumen sin conexión a la base de datos")

📊 RESUMEN DE ANÁLISIS SQL - TESLA STOCK

🔍 ESTADÍSTICAS GENERALES:
  • Período analizado: 2020-01-03 00:00:00 a 2023-12-29 00:00:00
  • Total de registros: 1,005 días
  • Cambio total de precio: 1602.4%
  • Retorno diario promedio: 0.307%
  • Retorno absoluto promedio: 3.077%
  • Días positivos: 540 (53.7%)
  • Días negativos: 464 (46.2%)
  • Volumen promedio: 133,216,673 acciones
  • Máxima ganancia diaria: 19.89%
  • Máxima pérdida diaria: -21.06%

🎯 INSIGHTS CLAVE DEL ANÁLISIS SQL:
  • Mejor mes para invertir: 1 (1.07%)
  • Peor mes históricamente: 12 (-0.26%)
  • Mejor día de la semana: 0 (1.13%)
  • Días de alto volumen con alta volatilidad: 53.3%

💡 RECOMENDACIONES BASADAS EN ANÁLISIS SQL:
  • Los análisis de rachas muestran patrones de momentum
  • La volatilidad se concentra en períodos específicos
  • Existe correlación entre volumen alto y movimientos significativos
  • Los patrones estacionales son detectables con SQL
  • Las consultas complejas revelan insights ocultos en l

## 8. Cerrar Conexión y Limpiar Recursos

In [28]:
# Limpiar recursos y cerrar conexión
if conn:
    # Opcional: Guardar algunos resultados clave
    try:
        import os
        if not os.path.exists('results'):
            os.makedirs('results')
        
        # Exportar tabla final con análisis SQL
        final_export_query = """
        SELECT 
            Date,
            ROUND(Close, 2) as Close,
            ROUND(Volume, 0) as Volume,
            ROUND(Daily_Return * 100, 3) as Daily_Return_Pct,
            Year,
            Month,
            DayOfWeek,
            Quarter
        FROM tesla_stock
        ORDER BY Date
        """
        
        export_df = pd.read_sql_query(final_export_query, conn)
        export_df.to_csv('results/tesla_sql_analysis_results.csv', index=False)
        
        print("💾 Resultados exportados a 'results/tesla_sql_analysis_results.csv'")
        
    except Exception as e:
        print(f"⚠️ Error al exportar resultados: {e}")
    
    # Cerrar conexión
    conn.close()
    print("✅ Conexión a la base de datos cerrada")
    print("🔒 Recursos limpiados exitosamente")
else:
    print("ℹ️ No hay conexión activa para cerrar")

print("\n🎊 ¡ANÁLISIS SQL DE TESLA COMPLETADO!")
print("📈 Datos analizados comprehensivamente con consultas SQL avanzadas")
print("🚀 ¡Listo para la siguiente fase del proyecto!")

💾 Resultados exportados a 'results/tesla_sql_analysis_results.csv'
✅ Conexión a la base de datos cerrada
🔒 Recursos limpiados exitosamente

🎊 ¡ANÁLISIS SQL DE TESLA COMPLETADO!
📈 Datos analizados comprehensivamente con consultas SQL avanzadas
🚀 ¡Listo para la siguiente fase del proyecto!
